In [1]:
################################################################################
%matplotlib notebook
import numpy as np
import pandas as pd
import theano
import theano.tensor as T
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, GRU
from typing import Tuple, Callable, List
from tracker import visuals, extractor, utils, metrics

Tensor = theano.tensor.Tensor

Using Theano backend.


In [2]:
################################################################################
order  = ["phi", "r", "z"]
frame  = pd.read_csv("data/sets/ACTS-SMALL-PREPARED.gz")
data   = extractor.extract_input(frame, order)
matrix = extractor.extract_output(frame, order)
input_shape  = data.shape[1:]
output_shape = matrix.shape[1:]
print(input_shape)
print(output_shape)
n = 2
visuals.display_matrices(data[n], matrix[n], order=order, noise=False)

(50, 3)
(50, 12)


,phi,r,z,A,B,C,D,E,F,G,H,I,J,K,padding
0,0.890630,31.959895,-195.88300,1,,,,,,,,,,,
1,0.896505,71.842967,-356.94300,1,,,,,,,,,,,
2,1.589236,31.959895,-333.45800,,1,,,,,,,,,,
3,1.643262,71.842967,-462.27500,,,1,,,,,,,,,
4,1.645726,31.959895,-240.43700,,,1,,,,,,,,,
5,2.020208,171.809967,-157.58100,,,,1,,,,,,,,
6,2.034692,115.818695,-119.58800,,,,1,,,,,,,,
7,2.045738,71.842967,-90.33470,,,,1,,,,,,,,
8,2.055850,31.959895,-63.94060,,,,1,,,,,,,,
9,3.443508,71.842967,-382.21600,,,,,1,,,,,,,


In [3]:
################################################################################
class LossFunctionCreator:
    """
    After initializing this class, it can be called to return a tensor 
    that is compatible with keras custom tensors.
    """
    def __init__(self,
            input_tensor : Tensor,
            output_shape : Tuple,
            order        : List[str],
            ) -> None:
        """ Initialize this loss creator. """
        self.__name__     = "LossFunctionCreator"
        self.input_tensor = input_tensor
        self.output_shape = output_shape
        self.order        = order
    
    def __call__(self,
            y_true : Tensor,
            y_pred : Tensor,
            ) -> Tensor:
        """ Return a loss tensor, given a prediction tensor and truth tensor. """
        tensor = None
        for i in range(output_shape[1]):
            pred_mask = self.get_track_mask(y_pred, 0)
            true_mask = self.get_track_mask(y_true, 0)
            pred_num_hits = T.sum(T.sum(pred_mask, axis=-1), axis=-1) / len(order)
            true_num_hits = T.sum(T.sum(true_mask, axis=-1), axis=-1) / len(order)
            pred_masked = pred_mask * self.input_tensor
            true_masked = true_mask * self.input_tensor
            pred_line = self.linear_regression(pred_masked, pred_num_hits)
            true_line = self.linear_regression(true_masked, true_num_hits)
            diff = (pred_line - true_line)**2
            avg  = T.mean(diff, axis=0)
            tensor = avg if tensor is None else tensor + avg
        return tensor / output_shape[1]
    
    def get_order_mask(self,
            output : Tensor,
            string : str,
            ) -> Tensor:
        """
        Return a mask such that when output is multiplied by this mask,
        only the column corresponding to the *string* category remains.
        """
        ones  = T.any(output, axis=-1)
        zeros = T.zeros_like(ones)
        stack = [zeros for _ in range(len(self.order) - 1)]
        stack.insert(self.order.index(string), ones)
        return T.stack(stack, axis=-1)
    
    def get_track_mask(self,
            output   : Tensor,
            track_id : int,
            ) -> Tensor:
        """
        Retrieve a tensor containing a mask such that if self.tensor_input
        was multiplied by the mask, the result would be a tensor containing
        the positions of all hits with the specified track_id.
        """
        cats = T.argmax(output, axis=-1)
        fill = T.fill(cats, track_id)
        eqs  = T.eq(cats, fill)
        mask = T.stack([eqs for _ in range(len(order))], axis=-1)
        return mask
    
    def linear_regression(self,
            tensor : Tensor,
            length : Tensor,
            ) -> Tensor:
        """
        Given a tensor, and the number of hits within the tensor,
        return the two parameters (m, b) of the least squares
        regression line with equation y=m*x+b.
        """
        r = T.roll(tensor * self.get_order_mask(tensor, "r"),
                   len(self.order) - self.order.index("r"), axis=-1)
        z = T.roll(tensor * self.get_order_mask(tensor, "z"),
                   len(self.order) - self.order.index("z"), axis=-1)
        a = z.sum() * (r**2).sum()  - r.sum() * (r * z).sum()
        b = length  * (r**2).sum()  - r.sum()**2
        c = length  * (r * z).sum() - r.sum() * z.sum()
        d = length  * (r**2).sum()  - r.sum()**2
        e = 2**(-10)  # ~0.001 epsilon to avoid division by 0.
        return T.stack([(c / (d + e)), (a / (b + e))])
    
A = T.tensor3("A")
B = T.tensor3("B")
C = T.tensor3("C")
D = LossFunctionCreator(A, output_shape, order)
E = D(B, C)
F = theano.function([A, B, C], E, on_unused_input='ignore')
print(theano.printing.pprint(E))
print(F(data, matrix, matrix[::-1]))

(((((((((((((Sum{axis=[0], acc_dtype=float64}(((join(TensorConstant{0}, ((((Sum{axis=[1], acc_dtype=int64}(Sum{axis=[2], acc_dtype=int64}(join(TensorConstant{2}, DimShuffle{0, 1, x}(eq(MaxAndArgmax{axis=(2,)}(C), fill(MaxAndArgmax{axis=(2,)}(C), TensorConstant{0}))), DimShuffle{0, 1, x}(eq(MaxAndArgmax{axis=(2,)}(C), fill(MaxAndArgmax{axis=(2,)}(C), TensorConstant{0}))), DimShuffle{0, 1, x}(eq(MaxAndArgmax{axis=(2,)}(C), fill(MaxAndArgmax{axis=(2,)}(C), TensorConstant{0})))))) / TensorConstant{3}) * Sum{acc_dtype=float64}((join(TensorConstant{2}, ((join(TensorConstant{2}, DimShuffle{0, 1, x}(eq(MaxAndArgmax{axis=(2,)}(C), fill(MaxAndArgmax{axis=(2,)}(C), TensorConstant{0}))), DimShuffle{0, 1, x}(eq(MaxAndArgmax{axis=(2,)}(C), fill(MaxAndArgmax{axis=(2,)}(C), TensorConstant{0}))), DimShuffle{0, 1, x}(eq(MaxAndArgmax{axis=(2,)}(C), fill(MaxAndArgmax{axis=(2,)}(C), TensorConstant{0})))) * A) * join(TensorConstant{2}, DimShuffle{0, 1, x}(fill(Any{2}(neq((join(TensorConstant{2}, DimShuffle{

In [ ]:
################################################################################
# Input Layer:
input_layer = Input(
    name  = "Input", 
    shape = input_shape,
)

# Hidden Layers:
model_layer = GRU(
    name  = "GRU 1",
    return_sequences=True,
    units = 256,
)(input_layer)
model_layer = GRU(
    name  = "GRU 2",
    return_sequences=True,
    units = 256,
)(model_layer)
model_layer = GRU(
    name  = "GRU 3",
    return_sequences=True,
    units = 256,
)(model_layer)

# Output Layer:
output_layer = Dense(
    name  = "Softmax",
    units = output_shape[1],
    activation = "softmax",
)(model_layer)

model = Model(inputs=input_layer, outputs=output_layer)
loss = LossFunctionCreator(input_layer, output_shape, order)
model.compile(loss=loss, optimizer="rmsprop")
model.summary()

In [ ]:
################################################################################
_ = model.fit(data, matrix, epochs=100, batch_size=3, verbose=2)
predictions = model.predict(data[0:2])
with open("output.txt", "w") as file:
    for prediction in predictions:
        for row in prediction:
            for num in row:
                file.write("{0:.2f} | ".format(num).replace("0.00", "    "))
            file.write("\n")
        file.write(("-" * 83) + "\n")

In [ ]:
################################################################################
# Input Layer:
input_layer = Input(
    name  = "Input", 
    shape = input_shape,
)

# Hidden Layers:
model_layer = GRU(
    name  = "GRU 1",
    return_sequences=True,
    units = 256,
)(input_layer)
model_layer = GRU(
    name  = "GRU 2",
    return_sequences=True,
    units = 256,
)(model_layer)
model_layer = GRU(
    name  = "GRU 3",
    return_sequences=True,
    units = 256,
)(model_layer)

# Output Layer:
output_layer = Dense(
    name  = "Softmax",
    units = output_shape[1],
    activation = "softmax",
)(model_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
model.summary()

In [ ]:
_ = model.fit(data, matrix, epochs=100, batch_size=3, verbose=2)
predictions = model.predict(data[0:2])
with open("output2.txt", "w") as file:
    for prediction in predictions:
        for row in prediction:
            for num in row:
                file.write("{0:.2f} | ".format(num).replace("0.00", "    "))
            file.write("\n")
        file.write(("-" * 83) + "\n")